## 1\. Penyiapan Lingkungan dan Inisialisasi Pipeline

Sel pertama dan kedua adalah tahap penyiapan. Mereka mengimpor pustaka yang diperlukan, mengonfigurasi variabel-variabel penting, dan menyiapkan konteks interaktif untuk menjalankan pipeline.

### Penjelasan

  * **Impor Pustaka**: Kode ini mengimpor `tensorflow`, `tfx`, dan `os`. Yang paling penting adalah `InteractiveContext`, yang memungkinkan komponen TFX dijalankan satu per satu di dalam notebook.
  * **Pengecekan Versi**: Mencetak versi TensorFlow (2.9.2) dan TFX (1.10.0) untuk memastikan reproduktifitas.
  * **Konfigurasi Path**:
      * `PIPELINE_NAME`: Memberi nama unik pada pipeline, yaitu "churn-prediction-pipeline".
      * `PIPELINE_ROOT`: Menentukan direktori utama tempat semua hasil (artefak) pipeline akan disimpan.
      * `DATA_ROOT`: Menunjuk ke folder `data` yang berisi dataset mentah.
  * **Inisialisasi Konteks**: `InteractiveContext(pipeline_root=PIPELINE_ROOT)` adalah "orkestrator" untuk sesi interaktif ini. Ia mengelola semua eksekusi komponen dan penyimpanan artefak serta metadata di dalam `PIPELINE_ROOT`. Peringatan `InteractiveContext metadata_connection_config not provided` adalah normal dan menunjukkan TFX akan membuat database metadata SQLite default di lokasi yang ditentukan.

-----

In [1]:
# Import libraries
import os
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tfx

print(f"TensorFlow version: {tf.__version__}")
print(f"TFX version: {tfx.__version__}")

# Inisialisasi path dan variabel
PIPELINE_NAME = "churn-prediction-pipeline"
PIPELINE_ROOT = os.path.join('.', 'dzikrimaulana87-pipeline')
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'metadata', 'metadata.db')
DATA_ROOT = "data"

# Inisialisasi InteractiveContext
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

TensorFlow version: 2.9.2
TFX version: 1.10.0


## 2\. Memuat Data dengan `CsvExampleGen`

Komponen ini bertanggung jawab untuk memuat data dari sumber eksternal (dalam hal ini, file CSV) ke dalam format yang dapat digunakan oleh komponen TFX lainnya.

### Penjelasan

  * **Tujuan**: `CsvExampleGen` membaca file CSV dari direktori yang ditentukan oleh `input_base` (`data`).
  * **Proses**: Komponen ini secara otomatis mengonversi data CSV menjadi format `TFRecord` yang efisien, dan membaginya menjadi dua set data (split): `train` dan `eval`. Pembagian ini penting untuk melatih dan mengevaluasi model secara terpisah.
  * **Output (Artefak)**: Hasilnya adalah artefak `Examples` yang disimpan di `.\dzikrimaulana87-pipeline\CsvExampleGen\examples\1`. Artefak ini berisi data `train` dan `eval` yang sudah siap untuk tahap selanjutnya.

-----

In [2]:
from tfx.components import CsvExampleGen

# Membuat instance ExampleGen
example_gen = CsvExampleGen(input_base=DATA_ROOT)

# Menjalankan komponen
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## 3\. Menghasilkan Statistik dengan `StatisticsGen`

Komponen ini menghitung statistik deskriptif untuk setiap fitur dalam dataset. Ini sangat penting untuk memahami data Anda.

### Penjelasan

  * **Tujuan**: `StatisticsGen` mengambil data dari `ExampleGen` dan menghasilkan statistik mendetail untuk setiap fitur.
  * **Input**: Menggunakan artefak `examples` dari `example_gen`.
  * **Output (Artefak & Visualisasi)**:
    1.  Menghasilkan artefak `ExampleStatistics` yang disimpan di `.\dzikrimaulana87-pipeline\StatisticsGen\statistics\2`.
    2.  `context.show()` menampilkan visualisasi interaktif **Facets** untuk split `train` dan `eval`. Visualisasi ini memungkinkan Anda untuk:
          * Melihat distribusi data numerik dan kategorikal.
          * Mengidentifikasi jumlah nilai yang hilang (*missing values*).
          * Membandingkan distribusi antara data `train` dan `eval` untuk mendeteksi *skew*.

-----

In [3]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)
context.run(statistics_gen)

# Tampilkan statistik (opsional tapi sangat dianjurkan)
context.show(statistics_gen.outputs['statistics'])

## 4\. Membuat Skema Data dengan `SchemaGen`

Berdasarkan statistik yang dihasilkan, `SchemaGen` membuat skema data. Skema ini berfungsi sebagai "kontrak" atau aturan yang harus dipatuhi oleh data di seluruh pipeline.

### Penjelasan

  * **Tujuan**: `SchemaGen` secara otomatis menyimpulkan tipe data, batasan, dan properti dari setiap fitur berdasarkan statistik.
  * **Input**: Menggunakan artefak `statistics` dari `statistics_gen`.
  * **Output (Artefak & Tampilan)**:
    1.  Artefak `Schema` disimpan di `.\dzikrimaulana87-pipeline\SchemaGen\schema\3`.
    2.  `context.show()` menampilkan tabel yang merangkum skema:
          * **Tipe Data**: Mengidentifikasi fitur sebagai `FLOAT`, `INT`, `STRING`, atau `BYTES`.
          * **Presence**: Menentukan apakah sebuah fitur harus selalu ada (`required`).
          * **Domain**: Untuk fitur `STRING`, skema mencantumkan semua nilai unik yang diharapkan (misalnya, 'gender' memiliki domain 'Female', 'Male').

-----

In [4]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)

# Tampilkan skema (opsional)
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Churn',STRING,required,,'Churn'
'Contract',STRING,required,,'Contract'
'Dependents',STRING,required,,'Dependents'
'DeviceProtection',STRING,required,,'DeviceProtection'
'InternetService',STRING,required,,'InternetService'
'MonthlyCharges',FLOAT,required,,-
'MultipleLines',STRING,required,,'MultipleLines'
'OnlineBackup',STRING,required,,'OnlineBackup'
'OnlineSecurity',STRING,required,,'OnlineSecurity'


,Values
Domain,
'Churn',"'No', 'Yes'"
'Contract',"'Month-to-month', 'One year', 'Two year'"
'Dependents',"'No', 'Yes'"
'DeviceProtection',"'No', 'No internet service', 'Yes'"
'InternetService',"'DSL', 'Fiber optic', 'No'"
'MultipleLines',"'No', 'No phone service', 'Yes'"
'OnlineBackup',"'No', 'No internet service', 'Yes'"
'OnlineSecurity',"'No', 'No internet service', 'Yes'"
'PaperlessBilling',"'No', 'Yes'"


## 5\. Memvalidasi Data dengan `ExampleValidator`

Komponen ini mencari anomali dan data yang tidak konsisten dengan membandingkan statistik data baru terhadap skema yang telah dibuat.

### Penjelasan

  * **Tujuan**: `ExampleValidator` memastikan kualitas dan konsistensi data. Ia akan mendeteksi masalah seperti tipe data yang salah, fitur yang hilang, atau nilai di luar domain yang diharapkan.
  * **Input**: Membutuhkan artefak `statistics` dan `schema`.
  * **Output (Artefak & Tampilan)**:
    1.  Menghasilkan artefak `ExampleAnomalies` di `.\dzikrimaulana87-pipeline\ExampleValidator\anomalies\4`.
    2.  Tampilan output menunjukkan **"No anomalies found."** untuk kedua split data. Ini berarti data Anda bersih dan konsisten dengan skema yang telah dibuat, sehingga aman untuk melanjutkan ke tahap berikutnya.

-----

In [5]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

# Tampilkan hasil validasi (opsional)
context.show(example_validator.outputs['anomalies'])

## 6\. Rekayasa Fitur dengan `Transform`

`Transform` adalah komponen yang sangat kuat untuk melakukan rekayasa fitur (*feature engineering*). Logika transformasi didefinisikan dalam file Python terpisah.

### Penjelasan

  * **Tujuan**: Melakukan pra-pemrosesan dan transformasi pada fitur untuk membuatnya lebih cocok untuk pelatihan model. Contohnya termasuk normalisasi fitur numerik, *one-hot encoding* fitur kategorikal, atau membuat fitur baru.
  * **Input**: Menggunakan data mentah (`examples`), skema (`schema`), dan sebuah file modul (`module_file`) yang berisi fungsi `preprocessing_fn`.
  * **Output (Artefak)**:
    1.  `transformed_examples`: Dataset yang telah ditransformasi, siap untuk digunakan oleh `Trainer`.
    2.  `transform_graph`: Artefak ini menyimpan logika transformasi. Ini sangat penting karena memastikan bahwa transformasi yang sama persis dapat diterapkan saat model digunakan untuk prediksi (*serving*), mencegah *training-serving skew*.

-----

In [6]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file='churn_transform.py'
)
context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: .\dzikrimaulana87-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\03f2a190a83d4ede878337034f9f5c68\assets


INFO:tensorflow:Assets written to: .\dzikrimaulana87-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\03f2a190a83d4ede878337034f9f5c68\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## 7\. Pelatihan Model dengan `Trainer`

Komponen ini mengambil data yang telah ditransformasi dan melatih sebuah model *machine learning*.

### Kode `Trainer`

### Penjelasan

  * **Tujuan**: Melatih model neural network menggunakan TensorFlow.
  * **Input**:
      * `module_file='churn_trainer.py'`: File ini mendefinisikan arsitektur model dan loop pelatihannya.
      * `transformed_examples`: Data latih dan evaluasi yang sudah diproses.
      * `transform_graph`: Untuk memastikan data diproses dengan benar di dalam model.
      * `schema`: Untuk informasi tentang fitur.
  * **Proses**: Kode ini menjalankan pelatihan selama 10 epoch. Output log menunjukkan metrik seperti `loss` dan `accuracy` untuk data latih dan validasi (`val_loss`, `val_accuracy`) pada setiap epoch.
  * **Output (Artefak)**: Artefak utamanya adalah `Model`, yang merupakan model terlatih yang disimpan di `.\dzikrimaulana87-pipeline\Trainer\model\6`.

-----

In [7]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file='churn_trainer.py', # Path ke file trainer
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100), # Sesuaikan jumlah step
    eval_args=trainer_pb2.EvalArgs(num_steps=50)
)
context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 gender_xf (InputLayer)         [(None, 1, 2)]       0           []                               
                                                                                                  
 Partner_xf (InputLayer)        [(None, 1, 2)]       0           []                               
                                                                                                  
 Dependents_xf (InputLayer)     [(None, 1, 2)]       0           []                               
                                                                                                  
 PhoneService_xf (InputLayer)   [(None, 1, 2)]       0           []                               
                                                                                              

                                                                  'lambda_7[0][0]',               
                                                                  'lambda_8[0][0]',               
                                                                  'lambda_9[0][0]',               
                                                                  'lambda_10[0][0]',              
                                                                  'lambda_11[0][0]',              
                                                                  'lambda_12[0][0]',              
                                                                  'lambda_13[0][0]',              
                                                                  'lambda_14[0][0]']              
                                                                                                  
 dense (Dense)                  (None, 64)           2880        ['concatenate[0][0]']            
          

INFO:tensorflow:Assets written to: .\dzikrimaulana87-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: .\dzikrimaulana87-pipeline\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## 8\. Evaluasi Kinerja Model dengan `Evaluator`

Setelah model dilatih, `Evaluator` menganalisis kinerjanya secara mendalam pada data evaluasi dan membandingkannya dengan model *baseline* (jika ada).

### Penjelasan

  * **Resolver**: Langkah pertama (`model_resolver`) adalah mencoba menemukan model yang sebelumnya telah "diberkati" (*blessed*) untuk digunakan sebagai pembanding. Pada eksekusi pertama, outputnya kosong karena belum ada model yang diberkati.
  * **Evaluator**:
      * **Tujuan**: Menilai apakah model yang baru dilatih (`candidate`) cukup baik untuk *deployment*.
      * **Input**: `Evaluator` memerlukan data evaluasi, model baru (`model`), dan model *baseline* (dari `model_resolver`).
      * **Konfigurasi (`eval_config`)**: Di sini, kita mendefinisikan aturan evaluasi:
          * `model_specs`: Menentukan `label_key` ('Churn\_xf') yang menjadi target prediksi.
          * `slicing_specs`: Menganalisis performa pada keseluruhan data dan juga pada irisan data spesifik (misalnya, per `gender`).
          * `metrics_specs`: Menetapkan metrik (seperti `BinaryAccuracy`, `AUC`) dan ambang batas (`threshold`). Model ini akan dianggap layak jika akurasinya di atas 0.75.
      * **Output (Artefak)**:
          * `evaluation`: Hasil analisis metrik yang mendalam.
          * `blessing`: Keputusan (berkati/tidak) berdasarkan apakah model memenuhi ambang batas pada `eval_config`. Artefak ini disimpan di `.\dzikrimaulana87-pipeline\Evaluator\blessing\8`.

-----

In [8]:
import tensorflow_model_analysis as tfma
from tfx.components import Evaluator
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.experimental.latest_blessed_model_resolver import LatestBlessedModelResolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class=LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=False)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [ ]:
# Evaluator dengan Konfigurasi Final
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_default', 
            label_key='Churn_xf'
        )
    ],
    slicing_specs=[
        # Slicing spec untuk melihat performa pada keseluruhan data
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['gender'])
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.75}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-10}
                    )
                )
            ),
            tfma.MetricConfig(class_name='Precision'),
            tfma.MetricConfig(class_name='Recall'),
            tfma.MetricConfig(class_name='AUC')
        ])
    ]
)

evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs.get('model'),
    eval_config=eval_config
)
context.run(evaluator)

## 9\. Mendorong Model ke Produksi dengan `Pusher`

`Pusher` adalah komponen terakhir dalam pipeline ini. Jika model telah "diberkati" oleh `Evaluator`, `Pusher` akan menyalinnya ke lokasi *deployment*.

### Penjelasan

  * **Tujuan**: Menyalin model yang telah divalidasi ke infrastruktur *serving*.
  * **Input**: `Pusher` bergantung pada dua artefak: `model` dari `Trainer` dan `model_blessing` dari `Evaluator`.
  * **Proses**: Komponen ini hanya akan berjalan jika artefak `model_blessing` ada dan berisi keputusan "diberkati" (*blessed*). Jika tidak, `Pusher` akan melewatkan proses ini.
  * **Output (Artefak)**: Jika berhasil, `Pusher` akan membuat artefak `PushedModel` dan menyalin file model ke direktori `push_destination`, yaitu `.\dzikrimaulana87-pipeline\serving_model`. Model ini sekarang siap untuk digunakan oleh aplikasi untuk membuat prediksi nyata.

In [ ]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2
# Pusher: Mendorong model ke direktori 'serving' jika lolos evaluasi
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(PIPELINE_ROOT, 'serving_model')
        )
    )
)
context.run(pusher)

## Menyusun dan Menjalankan Pipeline TFX dengan BeamDagRunner

Penjelasan Tiap Bagian:
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
Mengimpor runner untuk mengeksekusi pipeline menggunakan Apache Beam.

from tfx.orchestration.pipeline import Pipeline
Mengimpor kelas utama untuk menyusun pipeline TFX.

pipeline_name dan pipeline_root
Menentukan nama dan lokasi direktori utama pipeline. Semua artefak pipeline akan disimpan di pipeline_root.

metadata_path dan sqlite_metadata_connection_config()
Metadata pipeline disimpan dalam file SQLite di direktori pipeline. Metadata ini merekam semua run, artefak, dan hasil evaluasi.

components=components
Daftar komponen pipeline (seperti example_gen, trainer, evaluator, dll) yang sudah didefinisikan sebelumnya.

enable_cache=True
Mengaktifkan caching antar run untuk mempercepat proses jika tidak ada perubahan.

BeamDagRunner().run(...)
Menjalankan pipeline TFX secara lokal menggunakan DirectRunner dari Apache Beam

In [ ]:
components = [
    example_gen,
    statistics_gen,
    schema_gen,
    example_validator,
    transform,
    trainer,
    model_resolver,
    evaluator,
    pusher,
]

In [ ]:

from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.orchestration.pipeline import Pipeline
import os

# Konfigurasi pipeline
pipeline_name = 'churn_pipeline'
pipeline_root = os.path.join(os.getcwd(), 'dzikrimaulana87-pipeline', pipeline_name)

# Metadata SQLite file untuk menyimpan informasi pipeline
metadata_path = os.path.join(pipeline_root, 'metadata.sqlite')

from tfx.orchestration.metadata import sqlite_metadata_connection_config
metadata_connection_config = sqlite_metadata_connection_config(metadata_path)

# Jalankan pipeline menggunakan Apache Beam dengan DirectRunner
BeamDagRunner().run(
    Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        metadata_connection_config=metadata_connection_config,
        enable_cache=True,
    )
)